# Day 14

In [1]:
import os

In [16]:
input_file_path = os.path.join(".", "day14_sample.txt")
with open(input_file_path, 'r') as reader:
    input_data = reader.read()

## Part 1
Continuous sequence splicing

In [17]:
from collections import Counter

initial = None
instructions = {}

# Parse instructions
for line in input_data.split("\n"):
    if "->" in line:
        sequence, insertion = line.split(" -> ")
        instructions[sequence] = sequence[0] + insertion + sequence[1]
    elif initial is None:
        initial = line
    else:
        continue

# Process instructions using naive algorithm
steps = 10
word = initial
for step in range(steps):
    sequences = []

    # Process each 2-gram
    for i, (a, b) in enumerate(zip(word[:-1], word[1:])):
        ch = a + b
        if ch in instructions:
            token = instructions[ch]
        
            # Account for overlaps
            if i > 0:
                token = token[1:]

        else:
            token = ch
        sequences.append(token)

    # Generate new word
    word = "".join(sequences)

counts = Counter(word).most_common()
delta = counts[0][1] - counts[-1][1]
print(f"Part 1: delta between most and least common element is {delta:,}")

Part 1: delta between most and least common element is 1,588


## Part 2
Alternative algorithm to handle long mutations

In [18]:
initial = None
instructions = {}
graph = {}

# Parse instructions
for line in input_data.split("\n"):
    if "->" in line:
        sequence, insertion = line.split(" -> ")
        expansion = sequence[0] + insertion + sequence[1]
        instructions[sequence] = expansion
        graph[sequence] = {expansion[:2], expansion[1:]}
    elif initial is None:
        initial = line
    else:
        continue


initial

'NNCB'

In [19]:
graph

{'CH': {'BH', 'CB'},
 'HH': {'HN', 'NH'},
 'CB': {'CH', 'HB'},
 'NH': {'CH', 'NC'},
 'HB': {'CB', 'HC'},
 'HC': {'BC', 'HB'},
 'HN': {'CN', 'HC'},
 'NN': {'CN', 'NC'},
 'BH': {'BH', 'HH'},
 'NC': {'BC', 'NB'},
 'NB': {'BB', 'NB'},
 'BN': {'BB', 'BN'},
 'BB': {'BN', 'NB'},
 'BC': {'BB', 'BC'},
 'CC': {'CN', 'NC'},
 'CN': {'CC', 'CN'}}